**Table of contents**<a id='toc0_'></a>    
- [Capital Flow Schedule Module (cfsm)](#toc1_)    
  - [Extracting tickers obtained from ssm.ipynb](#toc1_1_)    
  - [Ranking the target stock list](#toc1_2_)    
  - [Obtaining the options and parsing](#toc1_3_)    
  - [Parsing contracts based on quantity and strike price](#toc1_4_)    
  - [Rank the contracts by descending premiums.](#toc1_5_)    
  - [Creating the CSV file for output.](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Capital Flow Schedule Module (cfsm)](#toc0_)
The cfsm creates a csv file using the target stock list previously generated in the stock screener module. Start dates and end dates are determined by the target stock’s selected derivative expiry date (PUTS, CALLS) and ex-dividend date. The csv file will be used to create the capital flow and the trade flow schedule.   

## <a id='toc1_1_'></a>[Extracting tickers obtained from ssm.ipynb](#toc0_)
The ticker symbols which we will use to query yahoo finance are located in a text file called subset_c.txt. Extracting them is trivial as we simply go line by line and append all the chars to the list.

In [ ]:
from pprint import pprint

ticker_names = []

with open(r'./subset_c.txt', 'r') as fp:
    for line in fp:
        x = line[:-1] # excluding \n char
        ticker_names.append(x)

pprint(ticker_names)

## <a id='toc1_2_'></a>[Ranking the target stock list](#toc0_)
Using the symbol names, we extract the data concerning the ticker from yfinance and then rank the target stock list by descending dividend yield.

In [ ]:
import yfinance as yf

tickers = list(yf.Tickers(ticker_names).tickers.values())

# pprint(tickers[0].info)
cnt = 0
for ticker in tickers:
    try:
        ticker.info['dividendYield']
    except KeyError:
        tickers.remove(ticker)

sorted_tickers = sorted(tickers, key=lambda x: x.info["dividendYield"], reverse=True)
print(sorted_tickers[0].info['dividendYield'])

## <a id='toc1_3_'></a>[Obtaining the options and parsing](#toc0_)
We obtain a list of all options of a given ticker and then for each target stock get the option chain with expiry dates more than 3 days but less than 15 days before the ex-div date 

We want a target stock with an ex div date that is relatively close so that the pay out is more likely.

In [ ]:
from datetime import datetime
tickers = []

for ticker in sorted_tickers:
    ticker_dates = {}
    ticker_dates["ticker"] = ticker
    ticker_dates["options"] = []
    exDividendDate = datetime.fromtimestamp(ticker.info["exDividendDate"]).date()
    ticker_dates["exDividendDate"] = exDividendDate
    for option in ticker.options:
        date = datetime.strptime(option, '%Y-%m-%d').date() 
        difference = exDividendDate - date
        if 3 < difference.days < 15:
            ticker_dates["options"].append(option)
    if len(ticker_dates["options"]) > 0:
        tickers.append(ticker_dates)

pprint(tickers)

## <a id='toc1_4_'></a>[Parsing contracts based on quantity and strike price](#toc0_)
We want to remove all contracts with an open interest less than 1000 contracts and with a strike price greater than or less than 2% of the current market price. 

The closer the market price to the strike price the better odds the contract will pay out after the ex-dividend date which is an outcome we desire.  

In [ ]:
import pandas as pd

for ticker in tickers:
    ticker['contracts'] = pd.DataFrame()
    market_price = ticker["ticker"].history()['Close'].iloc[-1]
    # print(f"MARKET PRICE: {market_price}")
    for option in ticker["options"]:
        chain = ticker["ticker"].option_chain(option)
        puts_calls = pd.concat([chain.puts, chain.calls])
        puts_calls.query('`openInterest` > 1000', inplace=True)
        puts_calls.query('`strike` > (@market_price * 0.98)', inplace=True)
        puts_calls.query('`strike` < @market_price * 1.02', inplace=True)
        ticker['contracts'] = pd.concat([ticker['contracts'], puts_calls])
    
pprint(tickers)



## <a id='toc1_5_'></a>[Rank the contracts by descending premiums.](#toc0_)

We rank the contracts per ticker by descending premiums so that the best contracts appear first in the csv file.

In [ ]:
for ticker in tickers:
    ticker['contracts'].sort_values('impliedVolatility')

pprint(tickers)

## <a id='toc1_6_'></a>[Creating the CSV file for output.](#toc0_)

We simply take all the fields given in yfinance and the ticker symbol + the ex-dividend date for easy interpretation of the results and further analysis.

In [ ]:
import csv

columns = [ 
    "symbol", "exDividendDate",
    "contractSymbol", "lastTradeDate", "strike",
    "lastPrice", "bid", "ask",
    "change", "percentChange", "volume",
    "openInterest", "impliedVolatility",
    "inTheMoney", "contractSize", "currency"
]
with open ('output.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    csv_writer.writerow(columns)
    for ticker in tickers:
        row = []
        for contract in ticker['contracts'].values:
            row = [ticker['ticker'].info["symbol"], ticker['exDividendDate']]  
            row.extend(contract)
            csv_writer.writerow(row)